In [1]:
!pip install -qU bitsandbytes transformers datasets accelerate loralib einops xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.5 MB/s eta 0:00:00


In [3]:
!pip install -qqq torch

In [4]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
notebook_login()

In [6]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [7]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


Adding LoRA config to add adapters to existing weights

In [9]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


Testing base model with no training

In [10]:
prompt = """
<human>: why in chess game, player took the following move - e4 e5 Nf3 Nc6 Bc4 h6 O-O d6 d3 a6 a4 Bg4 Be3 Qf6 h3 Bxf3 gxf3 Be7 f4 exf4 Bd2 f3 Bc3 Qg6+ Kh2

<assistant>:
""".strip()
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [11]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: why in chess game, player took the following move - e4 e5 Nf3 Nc6 Bc4 h6 O-O d6 d3 a6 a4 Bg4 Be3 Qf6 h3 Bxf3 gxf3 Be7 f4 exf4 Bd2 f3 Bc3 Qg6+ Kh2

<assistant>: The player took the move - e4 e5 Nf3 Nc6 Bc4 h6 O-O d6 d3 a6 a4 Bg4 Be3 Qf6 h3 Bxf3 gxf3 Be7 f4 exf4 Bd2 f3 Bc3 Qg6+ Kh2 to move the white queen to the g6 square.
User 
CPU times: user 9.24 s, sys: 1.45 s, total: 10.7 s
Wall time: 9.82 s


# **Starting finetuning**

Preparing dataset

In [12]:
import os
data = load_dataset("csv", data_files="5900_with_exp.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'progressive_moves', 'explanation'],
        num_rows: 5900
    })
})

In [14]:
data["train"][0]

{'Unnamed: 0': 0,
 'progressive_moves': 'd4',
 'explanation': "In a chess game, the players took the move d4 in order to open up the center of the board, establish a strong pawn presence in the center, and potentially gain control of key squares. By advancing the pawn to d4, the player gains more space for their pieces to maneuver and develop a strong position. This move also serves to challenge the opponent's position and create opportunities for tactical maneuvers in the midgame. Overall, the move d4 is a strategic decision aimed at gaining control and setting the stage for future play."}

In [15]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["progressive_moves"]}
<assistant>: {data_point["explanation"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [16]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/5900 [00:00<?, ? examples/s]

Finetuning

In [17]:
training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=False,
    save_total_limit=4,
    logging_steps=10,
    output_dir="experiments",
    save_strategy='epoch',
)

In [18]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.378500
20,1.995600
30,1.770800
40,1.591500
50,1.487000
60,1.466000
70,1.428900
80,1.380800
90,1.386900
100,1.395800


TrainOutput(global_step=738, training_loss=1.0653430368842147, metrics={'train_runtime': 4710.7702, 'train_samples_per_second': 1.252, 'train_steps_per_second': 0.157, 'total_flos': 7.396932179139072e+16, 'train_loss': 1.0653430368842147, 'epoch': 1.0})

In [19]:
model.save_pretrained("trained-model")

In [20]:
PEFT_MODEL = "adnaan525/capstone_cog_AI_5900_1"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/adnaan525/capstone_cog_AI_5900_1/commit/32998a860852e0f898824a3416f0306c471d7c65', commit_message='Upload model', commit_description='', oid='32998a860852e0f898824a3416f0306c471d7c65', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

testing the trained model

In [22]:
prompt = """
<human>: why in chess game, player took the following move - e4 e5 Nf3 Nc6 Bc4 h6 O-O d6 d3 a6 a4 Bg4 Be3 Qf6 h3 Bxf3 gxf3 Be7 f4 exf4 Bd2 f3 Bc3 Qg6+ Kh2

<assistant>:
""".strip()
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [23]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: why in chess game, player took the following move - e4 e5 Nf3 Nc6 Bc4 h6 O-O d6 d3 a6 a4 Bg4 Be3 Qf6 h3 Bxf3 gxf3 Be7 f4 exf4 Bd2 f3 Bc3 Qg6+ Kh2

<assistant>: In the given sequence of moves in the chess game, the last move Bc3 Qg6+ was played to capture the opponent's queen on g6. This move was made to gain material advantage by capturing the opponent's queen, which is a strong piece in the game. By capturing the queen, the player is also disrupting the opponent's pawn structure and potentially weakening their position on the board. This move also opens up the possibility of creating a strong attack on the opponent's king side. Overall, the move Bc3 Qg6+ was made to gain a strategic advantage in the game.
CPU times: user 8.61 s, sys: 104 ms, total: 8.72 s
Wall time: 8.7 s
